In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import mantis_core.rock_physics as manRP
import mantis_core.rock_physics.fluid as manFL
from mantis_core.rock_physics.fluid_presets import presets as manPResets
import numpy as np

In [ ]:
water = manRP.fluid.Fluid.from_presets(name="Water", temperature=30, pressure=25)
co2 = manRP.fluid.Fluid.from_presets(name="CarbonDioxide", temperature=30, pressure=25)
water_co2_mix = manRP.fluid.FluidMix(fluid1=water, fluid2=co2)

In [ ]:
in_situ = {"Vp": 3.8, "Vs": 1.8, "Rho": 2.5}

parameters = {
    "Phi": 0.2,
    "Km": 37.5,
    "fluid": mix1,
    "permeability": 1e-12,
    "patch_size": 0.0001,
    "bubble_radius": 0.0005,
    "eps": 0.1,
    "tau": np.log10(1 / 2 / np.pi / 35),
}

In [28]:
str(manRP.model_factory.ModelsBuild.models[0].__name__).split(".")[-1]

'sls'

In [39]:
{
    str(model.__name__).split(".")[-1]: {
        "name": model.Model.name,
        "type": model.Model.model_type,
        "reference": f"{model.Model.reference}",
    }
    for model in manRP.model_factory.ModelsBuild.models
}

{'sls': {'name': 'Standard Linear Solid model',
  'type': 'saturation-independent, frequency-dependent, isotropic',
  'reference': "Ursin, Björn, and Tommy Toverud. 'Comparison of seismic dispersion and attenuation models.' Studia geophysica et geodaetica 46 (2002): 293-320."},
 'gassmann': {'name': "Gassmann's model",
  'type': 'saturation-dependent, frequency-independent, isotropic',
  'reference': "\n        - Gassmann, F. (1951). \n            'Uber die Elastizitat poroser Medien.'\n            Vierteljahrsschrift der Naturforschenden Gesellschaft in Zurich, 96, 1-23.\n        \n        also see:\n        \n        - Berryman, James G. \n            'Origin of Gassmann's equations.' \n            Geophysics 64.5 (1999): 1627-1629.\n    "},
 'crm1d': {'name': 'Continuous 1-D Random Medium Model',
  'type': 'saturation-dependent, frequency-dependent, isotropic',
  'reference': '\n    Caspari, Eva, T. M. Müller, and Boris Gurevich.\n    `Time-lapse sonic logs reveal patchy CO2 saturat

In [ ]:
models = []

In [4]:

pars = {**in_situ, **parameters}

temp_model = manRP.models(identifier="white", **pars)
temp_model2 = manRP.models(identifier="crm1d", **pars)
temp_model2 = manRP.models(identifier="crm3d", **pars)

temp_model4 = manRP.models(identifier="gassmann", **pars)
white = manRP.models(identifier="white", **pars)
gassmann = manRP.models(identifier="gassmann", **pars)
chapman = manRP.models(identifier="chapman", **pars)


def mod(s):
    mix1.average = "Brie"
    mix1.patch_q = 0.2
    mix1.saturation = s
    return temp_model4.Cij()[0, 0]


def mod2(s):
    mix1.average = "Papageorgiou"
    mix1.patch_q = 0.2
    mix1.saturation = s
    return temp_model4.Cij()[0, 0]


def mod3(s):
    mix1.patch_q = 1
    mix1.saturation = s
    return temp_model4.Cij()[0, 0]


def mod4(s):
    mix1.patch_q = 1
    mix1.saturation = s
    return temp_model.Cij()[0, 0]


def mod5(s):
    mix1.patch_q = 1
    mix1.saturation = s
    return temp_model2.Cij()[0, 0]


def P_mods_by_saturation(s):
    mix1.saturation = s
    mix1.patch_q = 1
    gw_result = gassmann.Cij()[0, 0]
    mix1.average = "Brie"
    mix1.patch_q = 0.2
    brie = gassmann.Cij()[0, 0]
    mix1.average = "Papageorgiou"
    papa = gassmann.Cij()[0, 0]
    casp = caspari.Cij()[0, 0]
    whit = white.Cij()[0, 0]
    return [gw_result, brie, papa, casp, whit]


def P_attenuation_by_saturation(s):
    mix1.saturation = s
    mix1.patch_q = 1
    lam = lambda x: np.imag(x) / np.real(x)
    gw_result = lam(gassmann.Cij()[0, 0])
    mix1.average = "Brie"
    mix1.patch_q = 0.2
    brie = lam(gassmann.Cij()[0, 0])
    mix1.average = "Papageorgiou"
    chapman.eps = 0.005
    papa = lam(chapman.Cij(omega=-1)[0, 0])
    casp = -lam(caspari.Cij()[0, 0])
    whit = lam(white.Cij()[0, 0])
    return [brie, papa, casp, whit]


def P_mods_by_frequency(o):
    mix1.saturation = 0.75
    mix1.patch_q = 1
    lam = lambda x: np.imag(x) / np.real(x)
    gw_result = lam(gassmann.Cij()[0, 0])
    mix1.average = "Brie"
    mix1.patch_q = 0.2
    brie = gassmann.Cij()[0, 0]
    mix1.average = "Papageorgiou"
    chapman.eps = 0.005
    papa = chapman.Cij(omega=o - 1)[0, 0]
    casp = caspari.Cij(omega=o)[0, 0]
    whit = white.Cij(omega=o)[0, 0]
    return [brie, papa, casp, whit]


headers = [
    "Gassmann-Wood (1970)",
    "Brie et al. (1995)",
    "Papageorgiou et al. (2016)",
    "Caspari et al. (2011)",
    "White (1975)",
]
att_headers = [
    "Gassmann-Wood, Brie etc.",
    "Papageorgiou et al. (2016)",
    "Caspari et al. (2011)",
    "White (1975)",
]